In [1]:
###CONFIGURANDO TABELAS###

import pandas as pd
import holidays
from datetime import datetime, timedelta


# Caminho para salvar o arquivo no Google Drive
file_path = r'C:\Users\yurib\pjf-engenharia-de-dados\algo/'

In [2]:

###CARREGANDO ARQUIVOS###

datas = pd.read_csv(file_path+'bronze/dim_data.csv')
pedidos = pd.read_csv(file_path+'bronze/pedidos.csv')
localizacoes = pd.read_csv(file_path+'bronze/localizacoes.csv')

localizacoes = localizacoes.drop('Unnamed: 0', axis = 1)

C:\Users\yurib\AppData\Local\Temp\ipykernel_9396\3848116159.py:4: DtypeWarning: Columns (7,8,9,10,11,12,13,14,19) have mixed types. Specify dtype option on import or set low_memory=False.
  pedidos = pd.read_csv(file_path+'bronze/pedidos.csv')


In [3]:
###CRIANDO TRANSACAO###
dim_transacao = pedidos[['idcliente', 'SK_Pedidos', 'idvendedor', 'pagamento']]

In [4]:
###CRIANDO ENTREGAS###
entregas = pedidos[['SK_Pedidos', 'cepcliente', 'cepvendedor']]

#destino
temp = pd.merge(entregas, localizacoes, left_on='cepcliente', right_on='cep', how='left')
temp['en_destino'] = temp['SK_Localizacoes']
temp = temp.drop('SK_Localizacoes', axis = 1)
#origem
temp = pd.merge(temp, localizacoes, left_on='cepvendedor', right_on='cep', how='left')
temp['en_origem'] = temp['SK_Localizacoes']

entregas = temp[['SK_Pedidos', 'en_destino', 'en_origem', 'SK_Localizacoes']]

In [5]:
#Tratando dados#

pedidos['volume'] = pedidos['comprimento_cm'] * pedidos['altura_cm'] * pedidos['largura_cm']

pedidos['volume'] = pedidos['volume']/(100*100*100)
pedidos['massa'] = pedidos['peso_g']/1000

pedidos.rename(columns = {'id_produto':'idproduto'}, inplace = True)

#pedidos = pedidos[['idproduto', 'categoria', 'volume', 'massa']]

In [6]:

###CRIANDO PEDIDOS###

#df['data_apenas'] = df['datahora'].dt.date

# Selecionar e renomear as colunas
fato = pedidos[['SK_Pedidos', 'iditem', 'preco', 'frete', 'datacompra', 'datacliente', 'prazo', 'volume', 'massa']].copy()

# Converter as colunas de data para o tipo datetime
fato['datacompra'] = pd.to_datetime(fato['datacompra'])
fato['datacliente'] = pd.to_datetime(fato['datacliente'])
fato['prazo'] = pd.to_datetime(fato['prazo'])

# Calcular a diferença em dias para ped_diasentrega e ped_diasprazo
fato['ped_diasentrega'] = (fato['datacliente'] - fato['datacompra']).dt.days
fato['ped_diasprazo'] = (fato['prazo'] - fato['datacompra']).dt.days

final = fato[['SK_Pedidos', 'preco', 'frete', 'volume', 'massa', 'ped_diasentrega', 'ped_diasprazo', 'datacompra']]

In [7]:
#Tratando data
temp = final

temp['data_ref'] = temp['datacompra'].dt.date

temp = pd.merge(temp, datas, left_on='data_ref', right_on='dt_data', how='left')
temp['id_data'] = temp['sk_data']
#temp = temp.drop('SK_Localizacoes', axis = 1)

final = temp[['SK_Pedidos', 'preco', 'frete', 'volume', 'massa', 'ped_diasentrega', 'ped_diasprazo', 'id_data']]

C:\Users\yurib\AppData\Local\Temp\ipykernel_9396\734716246.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['data_ref'] = temp['datacompra'].dt.date


In [8]:

entregas.to_csv(file_path+'silver/dim_entregas.csv')
dim_transacao.to_csv(file_path+'silver/dim_transacao.csv')
localizacoes.to_csv(file_path+'silver/dim_localizacoes.csv')
final.to_csv(file_path+'silver/fato_pedidos.csv')
datas.to_csv(file_path+'silver/dim_data.csv')